# Machine vs Deep Learning

What is the difference? 

**Machine learning** is an algorithm that teaches a machine to think. More specifically, it is an algorith that can take in data, learn from that data and apply what they've learning to make informed decisions. 

**Deep learning** is machine learning. In other words deep learning is a subset of machine learning that mimics the way a human brain draws conclusions by inspiring from our neural network to create an artificial neural network. 

This section on deep learning uses *TensorFlow*, the most popular tool for deep learning and *Karis*, an API for specifying deep learning models. 

### Definition

__Tensor__: A matrix that can have a combination of any number of columns and rows.

## The Linear Unit

The fundamental component of a neural network is an individual neuron (or unit). An example with one input is the following :

In [1]:
from IPython.display import Image
Image(url="https://i.imgur.com/mfOlDR6.png")

    y = wx + b
    
where `x` is the input, its connection to the neuron is weight `w`. A special kind of weight is `b` which is called a bias: no input data (the 1 is for the image so the neuron receives the exact value of b). The point of the bias is to modify the outputs independently of its inputs.

The final output is `y`, which is also the slope-intercept equation from basic calculus. 

## TensorFlow and Keras

The easiest way to create a model in Keras (a popular library included in Tensorflow) is `keras.Sequential`. It creates a neural network as a stack of layers that each run a function on the input and in the end result in an output. The simplest example is `Dense` layer, which can take the inputs (input_shape) and output (units)

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

# Create a network with 1 linear unit
model = keras.Sequential([
    layers.Dense(units=1, input_shape=[3])
])

In terms of data, if we take a 2D array as an example (equivalent to an excel table), the input shape is the number of columns and the output shape is whatever we are trying to calculate. Above, we're taking 3 columns to calculate the value of a single feature.

Stacking dense layers with nothing in between adds no extra value and will never expand out of the 2D world. __Activation functions__ (a type of function) are applied to each layer to solve this problem. 

One of the most common activation function is the __rectified linear unit (ReLU)__. This function sets all negative values of the output to 0, which will put a _bend_ in the data and move the result away from simple lines.

In [3]:
from IPython.display import Image
Image(url="https://i.imgur.com/aeIyAlF.png", width=400)

In [4]:
from IPython.display import Image
Image(url="https://i.imgur.com/Y5iwFQZ.png", width=400)

In a complete and fully connected neural network, each input is pushed through each neuron. Since we don't directly see the output of the middle layers, they are often called hidden layers. The above network can be written as

In [ ]:
from tensorflow.keras import layers

model = keras.Sequential([
    # the hidden ReLU layers
    layers.Dense(units=4, activation='relu', input_shape=[2]),
    layers.Dense(units=3, activation='relu'),
    # the linear output layer 
    layers.Dense(units=1),
])

- The first layer must always have the input_shape defined, and then the following layers must maintain the same shape.
- Not all layers have to have activation
- Having a final output layer as linear (meaning no activation function) is common in regression tasks where an arbitrary numeric value is trying to be predicted

## Training a Neural Network

Up until now, the networks have provided outputs without actually learning anything. To create a neural network that learns from itself, we provide it training data and test data. In supervised learning, the test data contains the answers, and this way the network calculates the difference between its prediction and the true answer (with a __loss function__) and changes its weights to reduce this difference (or loss). 

To adjust the weights of the network, an __optimizer__ is used. A typical optimizer is the stochastic gradient descent (SGD). The above is repeated until the loss can't be reduced any further. To summarize:

1. Sample some training data and run it through the network to make predictions.
2. Measure the loss between the predictions and the true values.
3. Finally, adjust the weights in a direction that makes the loss smaller.

View an example <a href="https://i.imgur.com/rFI1tIk.mp4">here</a>

Each iteration's sample of training data is called a __minibatch__ (or often just "batch"), while a complete round of the training data is called an __epoch__. The number of epochs you train for is how many times the network will see each training example.

The speed at which a neural network arrives at its best values is called the __learning rate__. A smaller learning rate means the network needs to see more minibatches before it arrives at the best values. As a result, these two properties have the largest effect on how the SGD training proceeds.

While many algorithms have to be manually fine tuned to achieve the best results, __Adam__ is an SGD that has an adaptive learning rate and is a great general purpose optimizer.

In [ ]:
model.compile(
    optimizer="adam",
    loss="mae",
)

history = model.fit(
    X_train, y_train,
    validation_data=(X_valid, y_valid),
    batch_size=256,
    epochs=10,
)

## Overfitting and Underfitting

When a model is learning, its input data is not only _signal_ (the true data and what helps the model learn for new data) but also _noise_ (outliers, random fluctuations and only true for the training data). The balance between these two types of data is what determines the fit of the model.

    Overfitting is when the loss is not as low as it could be because the model hasn't learned enough signal
    Underfitting is when the loss is not as low as it could be because the model learned too much noise
    
Often the validation loss can decrease and then later on begin to increase. One method to arrive at a model with the lowest validation loss is __early stopping__. 

In [17]:
from IPython.display import Image
Image(url="https://i.imgur.com/eP0gppr.png", width=400)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(
    min_delta=0.001, # minimium amount of change to count as an improvement
    patience=20, # how many epochs to wait before stopping
    restore_best_weights=True,
)

history = model.fit(
    X_train, y_train,
    validation_data=(X_valid, y_valid),
    batch_size=256,
    epochs=500,
    callbacks=[early_stopping], # put your callbacks in a list
    verbose=0,  # turn off training log
)

Another way to prevent models from learning from too much noise, is randomly removing neurons from the network at each step traning. In this way, the model is forced to learn more general patterns whose weight patterns tend to be more robust. This idea is called __dropout__.

## Binary Classification

Until now all examples have regression problems. Another type of problem is when all data is placed into one of two classes. Theses problems have to very important properties:

- __Accuracy__ which is the ration of correct predictions over total predictions
- __Cross-entropy__ is a distance from one probability distribution to another
    
Both of these functions require probabilities as inputs and often the __sigmoid activation__ is used to convert inputs to values between 0 and 1.